In [1]:
import sys
import os
import torch

# This should print the path to your C: drive venv
print(f"Python Executable: {sys.executable}")
print(f"Working Directory: {os.getcwd()}") 
print(f"CUDA Available: {torch.cuda.is_available()}")

# Path configuration
TOOLS_DIR = r"C:\Users\ruben\Documents\TrainingAI" # <--- POINTS BACK TO OLD FOLDER
print(f"Tools Directory set to: {TOOLS_DIR}")

Python Executable: r:\Files Ruben\GitRepos\DeepDiveV2AI\.venv\Scripts\python.exe
Working Directory: r:\Files Ruben\GitRepos\DeepDiveV2AI
CUDA Available: True
Tools Directory set to: C:\Users\ruben\Documents\TrainingAI


In [ ]:
import torch
import json
import os
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import TrainerCallback
from huggingface_hub import login

# --- PATH CONFIGURATION ---
# 1. Where are the tools? (llama.cpp and quantize.exe)
TOOLS_DIR = r"C:\Users\ruben\Documents\TrainingAI"

# 2. Where should the final result go?
BASE_OUTPUT_DIR = r"R:\Files Ruben\GitRepos\DeepDiveV2AI\TrainedAndMerged"

# --- VERSIONING LOGIC ---
if not os.path.exists(BASE_OUTPUT_DIR):
    os.makedirs(BASE_OUTPUT_DIR)

version_num = 1
while os.path.exists(os.path.join(BASE_OUTPUT_DIR, f"Version{version_num}")):
    version_num += 1

OUTPUT_VERSION_DIR = os.path.join(BASE_OUTPUT_DIR, f"Version{version_num}")
os.makedirs(OUTPUT_VERSION_DIR)
print(f"📂 Output Folder Created: {OUTPUT_VERSION_DIR}")

# --- SETTINGS ---
DATA_FILE = "lore_training_data.json"
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
NEW_MODEL_NAME = "Llama-3-8B-Gloom-Lore"
HF_TOKEN = "token" 
TARGET_LOSS = 0.6 

# --- STOPPING LOGIC ---
class StopAtLossCallback(TrainerCallback):
    def __init__(self, threshold):
        self.threshold = threshold
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            if logs["loss"] <= self.threshold:
                print(f"\n🛑 STOPPING EARLY! Loss ({logs['loss']}) hit target.")
                control.should_training_stop = True

login(token=HF_TOKEN)

📂 Output Folder Created: R:\Files Ruben\GitRepos\DeepDiveV2AI\TrainedAndMerged\Version5


In [4]:
# 1. Load Data
try:
    with open(DATA_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"Loaded {len(data)} examples from {os.path.abspath(DATA_FILE)}")
except FileNotFoundError:
    raise FileNotFoundError(f"Could not find {DATA_FILE} in {os.getcwd()}")

# 2. Format Data
dataset = Dataset.from_list(data)
def format_llama3_prompts(examples):
    texts = []
    for instruction, response in zip(examples['instruction'], examples['response']):
        prompt = (
            f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
            f"You are a gritty shopkeeper on a submarine in a sci-fi horror setting.<|eot_id|>"
            f"<|start_header_id|>user<|end_header_id|>\n\n"
            f"{instruction}<|eot_id|>"
            f"<|start_header_id|>assistant<|end_header_id|>\n\n"
            f"{response}<|eot_id|>"
        )
        texts.append(prompt)
    return {'text': texts}

dataset = dataset.map(format_llama3_prompts, batched=True)
print("Data formatted.")

Loaded 161 examples from r:\Files Ruben\GitRepos\DeepDiveV2AI\lore_training_data.json


Map:   0%|          | 0/161 [00:00<?, ? examples/s]

Data formatted.


In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, quantization_config=bnb_config, device_map="auto", token=HF_TOKEN
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
TARGET_LOSS = 0.6

peft_config = LoraConfig(
    lora_alpha=16, lora_dropout=0.1, r=64, bias="none", task_type="CAUSAL_LM"
)

training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=15,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=50,   # <--- CHANGED FROM 5 TO 50
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    callbacks=[StopAtLossCallback(TARGET_LOSS)]
)

print("Starting training...")
trainer.train()
print("Training finished.")

# Save adapters to the Version Folder
trainer.model.save_pretrained(os.path.join(OUTPUT_VERSION_DIR, NEW_MODEL_NAME))
tokenizer.save_pretrained(os.path.join(OUTPUT_VERSION_DIR, NEW_MODEL_NAME))
print(f"LoRA adapters saved to: {OUTPUT_VERSION_DIR}")

Map:   0%|          | 0/161 [00:00<?, ? examples/s]

r:\Files Ruben\GitRepos\DeepDiveV2AI\.venv\Lib\site-packages\trl\trainer\sft_trainer.py:323: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128009}.


Starting training...


Step,Training Loss
50,2.550800
100,1.810700
150,1.721000
200,1.508100
250,1.347300
300,1.369000
350,1.149400
400,0.955200
450,0.974700
500,0.884000



🛑 STOPPING EARLY! Loss (0.5804) hit target.
Training finished.
LoRA adapters saved to: R:\Files Ruben\GitRepos\DeepDiveV2AI\TrainedAndMerged\Version5


In [7]:
from peft import PeftModel
import gc
import shutil

# 1. AGGRESSIVE CLEANUP
# We delete the training model from RAM to make space for the merge
try:
    del model
    del trainer
except NameError:
    pass 
gc.collect()
torch.cuda.empty_cache()

# 2. Define Overflow Folder
# If RAM fills up, Python will write temporary data here instead of crashing
offload_dir = os.path.join(OUTPUT_VERSION_DIR, "offload_temp")

print("Loading base model with Disk Offloading (Prevents Crash)...")
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",          # Use GPU first, then RAM, then Disk
    offload_folder=offload_dir, # <--- The Safety Net
    low_cpu_mem_usage=True,
    token=HF_TOKEN
)

print("Applying LoRA adapter...")
# Load the adapter we just saved in the version folder
adapter_path = os.path.join(OUTPUT_VERSION_DIR, NEW_MODEL_NAME)
model_to_merge = PeftModel.from_pretrained(base_model, adapter_path)

print("Merging weights...")
merged_model = model_to_merge.merge_and_unload()

# 3. Save with Sharding 
# Saves in 2GB chunks to prevent a RAM spike while writing to disk
output_merged_dir = os.path.join(OUTPUT_VERSION_DIR, "merged_model_temp")
print(f"Saving merged model to: {output_merged_dir}")

merged_model.save_pretrained(
    output_merged_dir, 
    safe_serialization=True, 
    max_shard_size="2GB" 
)
tokenizer.save_pretrained(output_merged_dir)

# 4. Clean up the offload folder immediately
if os.path.exists(offload_dir):
    shutil.rmtree(offload_dir)

# 5. Clean up the model from RAM to free up space for the next step
del base_model
del model_to_merge
del merged_model
gc.collect()

print("Merge & Save complete.")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading base model with Disk Offloading (Prevents Crash)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


Applying LoRA adapter...


r:\Files Ruben\GitRepos\DeepDiveV2AI\.venv\Lib\site-packages\accelerate\utils\modeling.py:1566: UserWarning: Current model requires 256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Merging weights...
Saving merged model to: R:\Files Ruben\GitRepos\DeepDiveV2AI\TrainedAndMerged\Version5\merged_model_temp


r:\Files Ruben\GitRepos\DeepDiveV2AI\.venv\Lib\site-packages\transformers\modeling_utils.py:3970: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

Merge & Save complete.


In [8]:
# Paths
llama_cpp_folder = os.path.join(TOOLS_DIR, "llama.cpp")
convert_script = os.path.join(llama_cpp_folder, "convert_hf_to_gguf.py")

# Points to the temp folder created in the previous cell
model_path = os.path.join(OUTPUT_VERSION_DIR, "merged_model_temp")
# Saves the heavy FP16 GGUF into the version folder
outfile_path = os.path.join(OUTPUT_VERSION_DIR, f"{NEW_MODEL_NAME}.fp16.gguf")   

if not os.path.exists(convert_script):
    print(f"Error: Could not find conversion script at: {convert_script}")
    print(f"Check if 'TOOLS_DIR' is correct in Cell 1.")
else:
    print(f"Using script: {convert_script}")
    print(f"Converting: {model_path}")
    !python "{convert_script}" "{model_path}" --outtype f16 --outfile "{outfile_path}"

Using script: C:\Users\ruben\Documents\TrainingAI\llama.cpp\convert_hf_to_gguf.py
Converting: R:\Files Ruben\GitRepos\DeepDiveV2AI\TrainedAndMerged\Version5\merged_model_temp


INFO:hf-to-gguf:Loading model: merged_model_temp
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00009.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00003-of-00009.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00004-of-00009.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00005-of-00009.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00009.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00007-of-00009.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00008-of-00009.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00006-of-00009.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00009-of-00009.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:to

In [10]:
import subprocess
import shutil
import os

# Define Paths
input_gguf = os.path.join(OUTPUT_VERSION_DIR, f"{NEW_MODEL_NAME}.fp16.gguf")
output_gguf = os.path.join(OUTPUT_VERSION_DIR, f"{NEW_MODEL_NAME}.Q4_K_M.gguf")
quantize_exe = os.path.join(TOOLS_DIR, "llama-quantize.exe")
temp_merged_folder = os.path.join(OUTPUT_VERSION_DIR, "merged_model_temp")

# 1. Pre-Flight Checks
if not os.path.exists(quantize_exe):
    print(f"❌ Error: Tool not found at: {quantize_exe}")
elif not os.path.exists(input_gguf):
    print(f"❌ Error: Input file not found: {input_gguf}")
    print("   👉 The previous 'Convert' step likely failed or didn't run.")
else:
    print(f"Quantizing...")
    print(f"   Input: {input_gguf}")
    
    try:
        # 2. Run Command (Using CWD=TOOLS_DIR to ensure it finds the CUDA DLLs)
        result = subprocess.run(
            [quantize_exe, input_gguf, output_gguf, "Q4_K_M"],
            cwd=TOOLS_DIR,        # <--- FIXES THE DLL LOADING ISSUE
            capture_output=True,  # Captures the logs
            text=True,
            check=True
        )
        
        print("\n✅ Quantization Complete.")
        
        # 3. Cleanup Logic
        print("🧹 Cleaning up massive temporary files...")
        
        if os.path.exists(input_gguf):
            os.remove(input_gguf)
            print(f"   - Deleted: {os.path.basename(input_gguf)}")
            
        if os.path.exists(temp_merged_folder):
            shutil.rmtree(temp_merged_folder)
            print(f"   - Deleted: merged_model_temp folder")
            
        print(f"\n🎉 SUCCESS! Final model saved to:\n{output_gguf}")

    except subprocess.CalledProcessError as e:
        print("\n❌ Quantization failed!")
        print("--- Error Details ---")
        print(e.stderr) # This prints the actual error message from the tool
        print("---------------------")

Quantizing...
   Input: R:\Files Ruben\GitRepos\DeepDiveV2AI\TrainedAndMerged\Version5\Llama-3-8B-Gloom-Lore.fp16.gguf

✅ Quantization Complete.
🧹 Cleaning up massive temporary files...
   - Deleted: Llama-3-8B-Gloom-Lore.fp16.gguf
   - Deleted: merged_model_temp folder

🎉 SUCCESS! Final model saved to:
R:\Files Ruben\GitRepos\DeepDiveV2AI\TrainedAndMerged\Version5\Llama-3-8B-Gloom-Lore.Q4_K_M.gguf
